Get the Manager Monthly Salary


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#Create dataframe:
#======================================================
#Employees Salary info
data1=[(100,"Raj",None,1,"01-04-23",50000),
       (200,"Joanne",100,1,"01-04-23",4000),(200,"Joanne",100,1,"13-04-23",4500),(200,"Joanne",100,1,"14-04-23",4020)]
schema1=["EmpId","EmpName","Mgrid","deptid","salarydt","salary"]
df_salary=spark.createDataFrame(data1,schema1)
display(df_salary)
#department dataframe
data2=[(1,"IT"),
       (2,"HR")]
schema2=["deptid","deptname"]
df_dept=spark.createDataFrame(data2,schema2)
display(df_dept)

EmpId,EmpName,Mgrid,deptid,salarydt,salary
100,Raj,null,1,01-04-23,50000
200,Joanne,100,1,01-04-23,4000
200,Joanne,100,1,13-04-23,4500
200,Joanne,100,1,14-04-23,4020


deptid,deptname
1,IT
2,HR


In [0]:
#Join with Dept
df=df_salary.join(df_dept,df_salary.deptid==df_dept.deptid,"left").drop(col("df_dept.deptid"))
#Convert the salary date column to date
df=df.withColumn("Salary_Date",to_date(col("salarydt"),'dd-MM-yy')).drop(col("salarydt"))
#Getting the Manager Name
df=df.alias('E').join(df.alias("M"),col("E.EmpId")==col("M.Mgrid"),"left").select(col('E.*'),col('M.EmpName').alias("Manager_Name"))
#Group by to get monthly Salary
df=df.groupBy('deptname','Manager_Name','EmpName',year('Salary_Date').alias("Sal_Year"),date_format('Salary_Date','MMMM').alias("Salary_Month")).agg(sum(col("salary")).alias("Total_Monthly_Salary"))


In [0]:
df.show()

+--------+------------+-------+--------+------------+--------------------+
|deptname|Manager_Name|EmpName|Sal_Year|Salary_Month|Total_Monthly_Salary|
+--------+------------+-------+--------+------------+--------------------+
|      IT|      Joanne|    Raj|    2023|       April|              150000|
|      IT|        null| Joanne|    2023|       April|               12520|
+--------+------------+-------+--------+------------+--------------------+

